In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from obspy import read, UTCDateTime
from scipy.signal import spectrogram
from sklearn.metrics import confusion_matrix, classification_report

catalog = pd.read_csv("complete_catalog.csv")
stream = read("complete_stream.mseed")

def apply_lowpass_filter(signal):
    st_low_pass = signal.copy()
    st_low_pass.filter("lowpass", freq=1)  
    return st_low_pass

def signal_to_spectrogram(signal, sampling_rate):
    f, t, Sxx = spectrogram(signal, fs=sampling_rate)
    return Sxx


def get_class_weights(y):
    total = len(y)
    class_0_weight = (1 / np.sum(y == 0)) * (total / 2.0)
    class_1_weight = (1 / np.sum(y == 1)) * (total / 2.0)
    return {0: class_0_weight, 1: class_1_weight}

sample_size = 1022


def load_and_process_data(catalog, stream, max_len=4000, sample_size=sample_size):
    X = []
    y = []
    
    
    catalog_sample = catalog.sample(n=sample_size, random_state=42)
    
    for index, row in catalog_sample.iterrows():
        start_time = UTCDateTime(row['starttime'])
        end_time = UTCDateTime(row['endtime'])
        label = row['class']  

        
        signal = stream.slice(starttime=start_time, endtime=end_time)
        if len(signal) == 0:
            continue
        signal_filtered = apply_lowpass_filter(signal)
        signal_data = np.hstack([tr.data for tr in signal_filtered])

        
        sampling_rate = signal[0].stats.sampling_rate

        
        if len(signal_data) > max_len:
            signal_data = signal_data[:max_len]
        elif len(signal_data) < max_len:
            signal_data = np.pad(signal_data, (0, max_len - len(signal_data)), 'constant')

        
        Sxx = signal_to_spectrogram(signal_data, sampling_rate)
        X.append(Sxx)
        y.append(label)

    X = np.array(X)
    y = np.array(y)
    
    if len(X.shape) == 3:  
        X = np.expand_dims(X, axis=-1)  
    
    return X, y

X, y = load_and_process_data(catalog, stream, sample_size=sample_size)

scaler = StandardScaler()


X_flat = X.reshape(X.shape[0], -1)

#
X_scaled = scaler.fit_transform(X_flat)


X_scaled = X_scaled.reshape(X.shape)


y_cat = to_categorical(y)


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_cat, test_size=0.2, random_state=42)


class_weights = get_class_weights(y)


def build_unet_autoencoder(input_shape):
    inputs = layers.Input(shape=input_shape)

    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    p1 = layers.MaxPooling2D((2, 2), padding='same')(c1)

    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    p2 = layers.MaxPooling2D((2, 2), padding='same')(c2)

    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p2)
    

    u1 = layers.UpSampling2D((2, 2))(c3)
    c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u1)

    u2 = layers.UpSampling2D((2, 2))(c4)
    c5 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u2)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid', padding='same')(c5)


    output_shape = outputs.shape[1:3]
    input_shape = inputs.shape[1:3]
    if output_shape != input_shape:
        crop_height = output_shape[0] - input_shape[0]
        crop_width = output_shape[1] - input_shape[1]
        if crop_height > 0 or crop_width > 0:
            outputs = layers.Cropping2D(((0, crop_height), (0, crop_width)))(outputs)

    autoencoder = models.Model(inputs, outputs)
    return autoencoder


input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
autoencoder = build_unet_autoencoder(input_shape)
optimizer = Adam(learning_rate=0.001)
autoencoder.compile(optimizer=optimizer, loss='mse')


history = autoencoder.fit(X_train, X_train, epochs=3, batch_size=16, validation_data=(X_test, X_test))

decoded_signals = autoencoder.predict(X_test)

print("Evaluación del autoencoder en datos de prueba:")
autoencoder.evaluate(X_test, X_test)

def build_classifier(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
    model.add(layers.MaxPooling2D((2, 2), padding='same'))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), padding='same'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='sigmoid')) 
    return model

classifier_optimizer = Adam(learning_rate=0.001)

classifier = build_classifier(input_shape)
classifier.compile(optimizer=classifier_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

classifier.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test), class_weight=class_weights)


print("Evaluación del clasificador en datos de prueba:")
classifier.evaluate(X_test, y_test)


y_pred = np.argmax(classifier.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)
cm = confusion_matrix(y_true, y_pred)
report = classification_report(y_true, y_pred)

print("Matriz de confusión:\n", cm)
print("Reporte de clasificación:\n", report)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score


y_pred_test = classifier.predict(X_test)
y_pred_train = classifier.predict(X_train)

y_pred_classes_test = np.argmax(y_pred_test, axis=1)
y_true_classes_test = np.argmax(y_test, axis=1)

y_pred_classes_train = np.argmax(y_pred_train, axis=1)
y_true_classes_train = np.argmax(y_train, axis=1)

precision_test = precision_score(y_true_classes_test, y_pred_classes_test, average=None)
recall_test = recall_score(y_true_classes_test, y_pred_classes_test, average=None)
f1_test = f1_score(y_true_classes_test, y_pred_classes_test, average=None)


precision_train = precision_score(y_true_classes_train, y_pred_classes_train, average=None)
recall_train = recall_score(y_true_classes_train, y_pred_classes_train, average=None)
f1_train = f1_score(y_true_classes_train, y_pred_classes_train, average=None)


report_test = classification_report(y_true_classes_test, y_pred_classes_test, target_names=['No Sismo', 'Sismo'])
conf_matrix_test = confusion_matrix(y_true_classes_test, y_pred_classes_test)


report_train = classification_report(y_true_classes_train, y_pred_classes_train, target_names=['No Sismo', 'Sismo'])
conf_matrix_train = confusion_matrix(y_true_classes_train, y_pred_classes_train)




print("\nMétricas para el conjunto de ENTRENAMIENTO:\n")
print("Reporte de Clasificación:\n", report_train)
print("Matriz de Confusión:\n", conf_matrix_train)


print("Métricas para el conjunto de PRUEBA:\n")
print("Reporte de Clasificación:\n", report_test)
print("Matriz de Confusión:\n", conf_matrix_test)



